# Feature Engineering

In [19]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('../Data/datasets/samoilovmikhail/simulated-refrigerator-fault-diagnosis-dataset/versions/1/fridge_fault_timeseries_dataset.csv')

In [3]:
df.columns

Index(['T_amb', 'T_set', 'T_cab', 'T_evap_sat', 'T_cond_sat', 'P_suc_bar',
       'P_dis_bar', 'N_comp_Hz', 'SH_K', 'P_comp_W', 'Q_evap_W', 'COP',
       'frost_level', 'T_cab_meas', 'valve_open', 'time_min', 'run_id',
       'fault', 'fault_id', 'door_open', 'defrost_on'],
      dtype='object')

------

## Physical Aware Derived Feature

#### Temperature Difference

In [4]:
df['delta_cab'] = df['T_cab_meas'] - df['T_set']
df['delta_cond_evap'] = df['T_cond_sat'] - df['T_evap_sat']
df['delta_amb_cab'] = df['T_amb'] - df['T_cab_meas']

#### Pressure Ratio

In [5]:
df['pressure_ratio']=df['P_dis_bar']/(df['P_suc_bar']+1e-6) 

#### Cooling Efficiency proxy

In [6]:
df['cooling_efficiency'] = df['Q_evap_W'] / (df['P_comp_W'] + 1e-6)


---

## Temporal behavior features

#### Rolling Stats.

In [7]:
window = 30
for col in ['COP','P_dis_bar','P_suc_bar','N_comp_Hz']:
    df[f'{col}_rolling_mean'] = df[col].rolling(window=window).mean()
    df[f'{col}_rolling_std'] = df[col].rolling(window=window).std() 

#### Rate of Change 

In [8]:
for col in['COP','P_dis_bar','P_suc_bar','N_comp_Hz']:
    df[f'{col}_diff'] = df[col].diff()

----

## Event Aware FE

#### Defrost-aware flags

In [9]:
df['is_defrost_window']=(
    df['defrost_on']
    .rolling(10)
    .max()
)

In [10]:
df['door_stress'] = df['door_open'].rolling(10).sum()

---------

# 

In [11]:
df = df.dropna().reset_index(drop=True)

In [12]:
feature_cols = [
    col for col in df.columns
    if col not in ['fault', 'fault_id', 'run_id', 'time_min']
]

corr = df[feature_cols].corr()

fig = px.imshow(
    corr,
    color_continuous_scale='RdBu',
    zmin=-1, zmax=1,
    title='Feature Correlation Matrix (Engineered Features)'
)
fig.show()


In [13]:
corr = df[feature_cols].corr()

In [14]:
corr

,T_amb,T_set,T_cab,T_evap_sat,T_cond_sat,P_suc_bar,P_dis_bar,N_comp_Hz,SH_K,P_comp_W,...,P_suc_bar_rolling_mean,P_suc_bar_rolling_std,N_comp_Hz_rolling_mean,N_comp_Hz_rolling_std,COP_diff,P_dis_bar_diff,P_suc_bar_diff,N_comp_Hz_diff,is_defrost_window,door_stress
T_amb,1.000000,-0.007259,0.446131,0.207273,0.955829,0.354551,0.943832,0.169196,0.009252,0.224078,...,0.350312,0.074148,0.205010,0.003670,-0.000201,0.017056,0.006590,0.000785,0.001768,0.002769
T_set,-0.007259,1.000000,0.053134,0.054557,-0.009795,0.044817,-0.008151,-0.030270,0.005757,-0.073603,...,0.044582,-0.004155,-0.037146,0.001127,0.000123,0.000077,0.000502,-0.000026,0.001365,0.002897
T_cab,0.446131,0.053134,1.000000,0.872201,0.361532,0.979455,0.349337,0.450825,0.128381,-0.417705,...,0.976367,0.104831,0.514418,0.147956,0.021531,0.009088,-0.001111,0.022436,0.102244,0.019136
T_evap_sat,0.207273,0.054557,0.872201,1.000000,0.185767,0.921208,0.179131,0.457257,-0.081674,-0.570616,...,0.913591,0.065710,0.569656,0.122330,0.005267,0.002655,0.011262,0.005313,0.109564,0.008481
T_cond_sat,0.955829,-0.009795,0.361532,0.185767,1.000000,0.280387,0.991232,0.230344,-0.122663,0.352120,...,0.286110,0.032100,0.212306,-0.043571,0.016541,0.020772,-0.010578,0.016998,-0.075062,0.003762
P_suc_bar,0.354551,0.044817,0.979455,0.921208,0.280387,1.000000,0.272077,0.395618,0.116502,-0.534904,...,0.988654,0.093144,0.499832,0.154635,0.007698,0.004308,0.031652,0.008257,0.139517,0.013498
P_dis_bar,0.943832,-0.008151,0.349337,0.179131,0.991232,0.272077,1.000000,0.218091,-0.121816,0.352386,...,0.277565,0.034408,0.199761,-0.041993,0.015770,0.058692,-0.010355,0.016365,-0.071895,0.003579
N_comp_Hz,0.169196,-0.030270,0.450825,0.457257,0.230344,0.395618,0.218091,1.000000,-0.071055,0.288543,...,0.476525,-0.315523,0.667150,-0.397214,0.137850,0.036699,-0.130313,0.134803,-0.630274,0.008152
SH_K,0.009252,0.005757,0.128381,-0.081674,-0.122663,0.116502,-0.121816,-0.071055,1.000000,-0.270223,...,0.117404,0.014881,-0.087098,-0.013536,-0.000194,0.000122,0.000275,-0.000174,-0.004981,0.002268
P_comp_W,0.224078,-0.073603,-0.417705,-0.570616,0.352120,-0.534904,0.352386,0.288543,-0.270223,1.000000,...,-0.472571,-0.213350,-0.054659,-0.313314,0.099855,0.028872,-0.107029,0.097977,-0.491079,0.006250


In [15]:
df.columns 


Index(['T_amb', 'T_set', 'T_cab', 'T_evap_sat', 'T_cond_sat', 'P_suc_bar',
       'P_dis_bar', 'N_comp_Hz', 'SH_K', 'P_comp_W', 'Q_evap_W', 'COP',
       'frost_level', 'T_cab_meas', 'valve_open', 'time_min', 'run_id',
       'fault', 'fault_id', 'door_open', 'defrost_on', 'delta_cab',
       'delta_cond_evap', 'delta_amb_cab', 'pressure_ratio',
       'cooling_efficiency', 'COP_rolling_mean', 'COP_rolling_std',
       'P_dis_bar_rolling_mean', 'P_dis_bar_rolling_std',
       'P_suc_bar_rolling_mean', 'P_suc_bar_rolling_std',
       'N_comp_Hz_rolling_mean', 'N_comp_Hz_rolling_std', 'COP_diff',
       'P_dis_bar_diff', 'P_suc_bar_diff', 'N_comp_Hz_diff',
       'is_defrost_window', 'door_stress'],
      dtype='object')

In [30]:
FEATURES=["T_amb",
"T_evap_sat",
"P_dis_bar",
"P_suc_bar",
"P_comp_W",
"N_comp_Hz",
"delta_cond_evap",
"cooling_efficiency",
"P_dis_bar_rolling_mean",
"P_suc_bar_rolling_std",
"COP_diff",
"N_comp_Hz_diff",
"door_open",
"frost_level"]

df_model = df[FEATURES + ['run_id', 'fault_id']].dropna().reset_index(drop=True)

In [31]:
run_ids = df_model['run_id'].unique()

train_runs, val_runs = train_test_split(run_ids, test_size=0.2, random_state=42,shuffle=True)

train_df = df_model[df_model['run_id'].isin(train_runs)].reset_index(drop=True)
val_df = df_model[df_model['run_id'].isin(val_runs)].reset_index(drop=True)

In [35]:
train_df.to_csv('../Data/processed/train_data.csv', index=False)
val_df.to_csv('../Data/processed/val_data.csv', index=False)

In [ ]:

def create_windows(df, window_size=60, stride=15):
    X, y = [], []

    for run_id in df['run_id'].unique():
        run_data = df[df['run_id'] == run_id]

        values = run_data[FEATURES].values
        label = run_data['fault_id'].iloc[0]

        for start in range(0, len(values) - window_size, stride):
            end = start + window_size

            X.append(values[start:end])
            y.append(label)

    return np.array(X), np.array(y)


In [33]:
X_train, y_train = create_windows(train_df)
X_val, y_val     = create_windows(val_df)

print(X_train.shape)
print(y_train.shape)


(95679, 60, 14)
(95679,)
